<a href="https://colab.research.google.com/github/MeldaChen/Group-4/blob/main/%E4%B8%B2paddle_paddle_%E8%A9%95%E5%88%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!python3 -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 121.6 MB 38 kB/s 
     |████████████████████████████████| 385 kB 8.9 MB/s 


In [47]:
pip install "paddleocr>=2.0.1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 411 kB 5.0 MB/s 
     |████████████████████████████████| 5.6 MB 50.8 MB/s 
     |████████████████████████████████| 965 kB 45.3 MB/s 
     |████████████████████████████████| 87 kB 6.1 MB/s 
     |████████████████████████████████| 148 kB 53.5 MB/s 
     |████████████████████████████████| 2.2 MB 50.5 MB/s 
     |████████████████████████████████| 8.8 MB 51.1 MB/s 
     |████████████████████████████████| 619 kB 60.5 MB/s 
     |████████████████████████████████| 4.9 MB 45.4 MB/s 
     |████████████████████████████████| 399 kB 62.8 MB/s 
     |████████████████████████████████| 132 kB 70.2 MB/s 
     |████████████████████████████████| 204 kB 73.2 MB/s 
     |████████████████████████████████| 2.3 MB 44.8 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=33e9dc820b22f0ef268fd9e640fbe41edca5b4bcdfd0099ddec5fdd91bf4c8a5

In [53]:
    import cv2
    import numpy as np
    import IPython.display as dp
    from google.colab.patches import cv2_imshow
    from PIL import ImageFont, ImageDraw, Image
    from matplotlib import pyplot as plt 
    from paddleocr import PaddleOCR, draw_ocr

    m1 = cv2.imread('A036.jpg', 1)

    copy = m1.copy()

    m1_g = cv2.cvtColor(m1, cv2.COLOR_BGR2GRAY) # 手寫字轉灰階

    ret, m1_new = cv2.threshold(m1_g, 127, 255, cv2.THRESH_BINARY) # 手寫字轉黑底白字 

    erode = cv2.erode(copy, np.ones((30, 30))) #抓字

    gray = cv2.cvtColor(copy, cv2.COLOR_BGR2GRAY) #轉灰階

    blur = cv2.GaussianBlur(gray,(5,5),0)   #模糊化


    canny = cv2.Canny(blur, 70, 210)

    th, canny = cv2.threshold(canny, 10, 255, cv2.THRESH_BINARY)

    morph = cv2.morphologyEx(canny, cv2.MORPH_GRADIENT, np.ones((10, 10))) 
    p, t = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(p)== 1:
          x, y, w, h = cv2.boundingRect(p[0]) 
          cv2.rectangle(m1_new, (x,y), (x+w, y+h), (255, 255, 255), 2) #框線改白色
          capture_img = m1_new[y : y + h, x : x + w] #框起來的地方
          
          resize_capture = cv2.resize(capture_img, (50, 50), interpolation=cv2.INTER_AREA)

          cv2.imwrite("resize_capture.png", resize_capture) #存成 50*50 的手寫字
          resize_capture = cv2.imread("resize_capture.png", 1)
          # 手寫辨識--------------------------------
          # y_dict = {0:"者",1:"我",2:"國",3:"來",4:"面",5:"過",6:"為",7:"大",8:"成",9:"民"}
          # m = cv2.ml.KNearest_load('tch_knn.xml')   # 載入模型
          # img = cv2.imread("resize_capture.png", 0)
          # ret, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
          # img = cv2.resize(img,(50, 50))
          # img = img.astype(np.float32)    # 轉換格式
          # img = img.reshape(-1)          # 打散成一維陣列資料，轉換成辨識使用的格式
          # img = img.reshape(1,-1)
          # img = img/255

          # img_pre = m.predict(img)          # 進行辨識

          # text = y_dict[int(img_pre[1][0][0])]        # 取得辨識結果

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
          ocr = PaddleOCR(use_angle_cls=True, lang="ch")  # need to run only once to download and load model into memory
          img_path = 'A036.jpg'
          result = ocr.ocr(img_path, cls=True)
          for idx in range(len(result)):
              res = result[idx]
              for line in res:
                  capture_word=line[-1][0]
                  text = capture_word


          compare_word = np.full((50, 50, 3), (255, 255, 255), np.uint8) 
          compare_word = Image.fromarray(compare_word)
          ImageDraw.Draw(compare_word).text((0,0),
                      (text), #要換成練習的字
                      (0, 0, 0),
                      ImageFont.truetype("ThePeakFontBeta_V0_101.ttf", 50)  #要上傳字體
          )
          compare_word = np.array(compare_word)
          

          

          cv2.imwrite("compare.png", compare_word) # 生成的字存成 compare.png，要再取一次輪廓

          compare_img = cv2.imread('compare.png', 1) 

          copy = compare_img.copy()

          # erode = cv2.erode(copy, np.ones((30, 30)))  

          # blur = cv2.GaussianBlur(erode,(5,5),0) 

          canny = cv2.Canny(copy, 70, 210)

          th, canny = cv2.threshold(canny, 10, 255, cv2.THRESH_BINARY)

          morph = cv2.morphologyEx(canny, cv2.MORPH_GRADIENT, np.ones((10, 10))) 
          p, t = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
          x, y, w, h = cv2.boundingRect(p[0]) 
          cv2.rectangle(compare_img, (x,y), (x+w, y+h), (255, 255, 255), 2)  #框線改白色
          cv2.imwrite("compare2.png", compare_img[y : y + h, x : x + w]) #要再resize

          compare2_img = cv2.imread('compare2.png', 1)

          resize_compare = cv2.resize(compare2_img, (50, 50), interpolation=cv2.INTER_AREA)
          cv2.imwrite("resize_compare.png", resize_compare) #存成 50*50 的比對字
        
          # 用resize_capture.png 跟resize_compare.png 比對分數

          # 最簡單的以灰度直方圖作為相似比較的實現 

          image1 = cv2.imread('resize_compare.png')
          image2 = cv2.imread('resize_capture.png')

          # 先計算直方圖 
          # 幾個引數必須用方括號括起來 
          # 這裡直接用灰度圖計算直方圖，所以是使用第一個通道， 
          # 也可以進行通道分離後，得到多個通道的直方圖 
          # bins 取為16 
          image1 = cv2.resize(image1,(256,256)) 
          image2 = cv2.resize(image2,(256,256)) 
          hist1 = cv2.calcHist([image1],[0],None,[256],[0.0,255.0]) 
          hist2 = cv2.calcHist([image2],[0],None,[256],[0.0,255.0]) 

          # 計算直方圖的重合度 
          degree = 0 
          for i in range(len(hist1)): 
            if hist1[i] != hist2[i]: 
              degree = degree + (1 - abs(hist1[i]-hist2[i])/max(hist1[i],hist2[i])) 
            else: 
              degree = degree + 1 
          degree = degree/len(hist1) 
      
          print("您的分數是", int(degree*100), "分")

    else:
        print("請靠近文字重新拍攝再上傳")


[2022/12/02 06:13:30] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set

In [49]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
ocr = PaddleOCR(use_angle_cls=True, lang="ch")  # need to run only once to download and load model into memory
img_path = 'A012.jpg'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        capture_word=line[-1][0]
        print(capture_word)
        

# 显示结果
# from PIL import Image

[2022/12/02 06:09:29] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set